# Plot image

In [ ]:
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

object_points, image_points, ret, k_matrix, distortion, r_vecs, t_vecs = np.load("Calibration/cam_parameters_iPhone8.npy", allow_pickle=True)

# Load the image and undistort it using the calibration parameters
imageName = 'images/book.JPG'
image = cv2.imread(imageName)
undistorted_image = cv2.undistort(image, k_matrix, distortion)

fig, axs = plt.subplots(1,2,figsize=(12,6))
axs[0].set_axis_off()
axs[1].set_axis_off()
axs[0].imshow(image, aspect = 'auto')
axs[1].imshow(undistorted_image, aspect = "auto")
axs[0].set_title("image")
axs[1].set_title("undistorted image")

In [ ]:
##Using undistorted image important? 
##if important insert:
#image = undistorted_image

In [ ]:
imageWithEdges = cv2.imread(imageName)
gray = cv2.cvtColor(imageWithEdges, cv2.COLOR_RGB2GRAY)
#find optimum parameters for canny (edge detection)
v = np.mean(gray)
sigma = 0.33
cannyTh1 = int(max(0, (1.0 - sigma) * v))
cannyTh2 = int(min(255, (1.0 + sigma) * v))
edges = cv2.Canny(gray, cannyTh1, cannyTh2)

lines = cv2.HoughLinesP(edges, 1, np.pi/180, 30, maxLineGap=2500)
whiteImage = np.full((imageWithEdges.shape[0],imageWithEdges.shape[1],imageWithEdges.shape[2]),255,dtype = np.uint8)

for line in lines:
  x1, y1, x2, y2 = line[0]
  cv2.line(imageWithEdges, (x1, y1), (x2, y2), (0, 255, 0), 1)
  cv2.line(whiteImage, (x1, y1), (x2, y2), (0, 255, 0), 1)

fig, axs = plt.subplots(1,2,figsize=(12,3))
fig.suptitle("canny thresholds = "+str(cannyTh1)+" , "+str(cannyTh2))
axs[0].set_axis_off()
axs[0].imshow(imageWithEdges, aspect = 'auto')
axs[1].imshow(whiteImage, aspect = "auto")
axs[0].set_title("image with vanishing lines")
axs[1].set_title("vanishing lines")

---
# BEGIN SECTION MICHI

In [ ]:
def detect_vanishing_lines(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #find optimum parameters for canny (edge detection)
    v = np.mean(gray)
    sigma = 0.33
    cannyTh1 = int(max(0, (1.0 - sigma) * v))
    cannyTh2 = int(min(255, (1.0 + sigma) * v))
    edges = cv2.Canny(gray, cannyTh1, cannyTh2)

    lines = cv2.HoughLines(edges, 1, np.pi/180.0, 120, np.array([]))
    return lines

In [ ]:
def extend_line_starting_points(tuple):
    x1,y1 = tuple[0]
    x2,y2 = tuple[1]

    # Calculate the slope
    slope = (y2 - y1) / (x2 - x1)

    # Calculate the y-intercept
    y_intercept = y1 - slope * x1

    #extend the line
    x1 = x1*-1000
    y1 = np.int32 (slope * x1 + y_intercept)
    x2 = x2*1000
    y2 = np.int32(slope * x2 + y_intercept)

    return ((x1,y1), (x2,y2))

In [ ]:
points = []

def on_mouse_click(event, x, y, flags, param):
    # if the left mouse button is clicked
    if event == cv2.EVENT_LBUTTONDOWN:
        # append the coordinates of the clicked point to the list
        points.append((x, y))


def let_user_draw_vanishing_lines_manually(lines):
    points.clear()
    print("Do you want to add a new line? (y/n)")
    if input() == 'y':
        #print the image and let the user click on the position. The user should select points which gets then appended as a tuple to lines
        # Create a window to display the image
        cv2.namedWindow('image')

        # Set the mouse callback function for the window
        cv2.setMouseCallback('image', on_mouse_click)

        while True:
            # Display the image in a new window
            cv2.imshow('image', imageWithEdgesOptimized)

            # Check if the user pressed the 'q' key or already 2 points selected
            if cv2.waitKey(1) & 0xFF == ord('q') or len(points) == 2:
                break

        # Destroy the window and exit the program
        cv2.destroyAllWindows()

        x1,y1 = points[0]
        x2,y2 = points[1]

        temp = imageWithEdgesOptimized.copy()
        #let user check if he/she wants to keep the line
        cv2.line(temp,(x1,y1),(x2,y2),(0,255,0),1)
        plt.imshow(temp)
        #wait 2 seconds
        plt.pause(2)
        plt.close()
        print("Do you want to keep this line? (y/n)")
        if input() == 'y':
            ((x1,y1),(x2,y2)) = extend_line_starting_points(((x1,y1),(x2,y2)))
            cv2.line(imageWithEdgesOptimized,(x1,y1),(x2,y2),(0,255,0),1)
            print("To which axis does the line correspond? (x/y/z)")
            answer = input()
            if answer == 'x':
                lines['x'].append(((x1,y1),(x2,y2)))
            if answer == 'y':
                lines['y'].append(((x1,y1),(x2,y2)))
            if answer == 'z':
                lines['z'].append(((x1,y1),(x2,y2)))
        return True
    else:
        return False

In [ ]:
image = cv2.imread(imageName)
lines = detect_vanishing_lines(image)
imageWithEdgesOptimized = image.copy()

#declare an axis dictionary to store the start and end points of the lines
lineStartEndPointsAxisMap = {'x': [], 'y': [], 'z': []}

for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 10000*(-b))
    y1 = int(y0 + 10000*(a))
    x2 = int(x0 - 10000*(-b))
    y2 = int(y0 - 10000*(a))
    temp = image.copy()
    #let user check if he/she wants to keep the line
    cv2.line(temp,(x1,y1),(x2,y2),(0,255,0),1)
    plt.imshow(temp)
    #wait 1 second
    plt.pause(1)
    plt.close()
    print("Do you want to keep this line? (y/n)")
    if input() == 'y':
        cv2.line(imageWithEdgesOptimized,(x1,y1),(x2,y2),(0,255,0),1)
        print("To which axis does the line correspond? (x/y/z)")
        answer = input()
        if answer == 'x':
            lineStartEndPointsAxisMap['x'].append(((x1,y1),(x2,y2)))
        if answer == 'y':
            lineStartEndPointsAxisMap['y'].append(((x1,y1),(x2,y2)))
        if answer == 'z':
            lineStartEndPointsAxisMap['z'].append(((x1,y1),(x2,y2)))
        

while let_user_draw_vanishing_lines_manually(lineStartEndPointsAxisMap):
    pass

fig, axs = plt.subplots(1,2,figsize=(12,3))
axs[0].set_axis_off()
axs[1].set_axis_off()
axs[0].imshow(imageWithEdges, aspect = 'auto')
axs[1].imshow(imageWithEdgesOptimized, aspect = "auto")
axs[0].set_title("image with vanishing lines")
axs[1].set_title("image with optimized vanishing lines")

In [ ]:
extendedLines = image.copy()

for key, element in lineStartEndPointsAxisMap.items():
    for e in element:
        cv2.line(extendedLines,e[0], e[1],(0,255,0),1)

plt.imshow(extendedLines, aspect = "auto")

#### Determine Vanishing Points

In [ ]:
from shapely.geometry import LineString

vanishingPoints = {'x':(), 'y':(), 'z':()}

for key, element in lineStartEndPointsAxisMap.items():
    #calculate the intersection of the first 2 lines (the others should also intersect at the same point)
    t1 = element[0]
    t2 = element[1]
    #create two lines
    line1 = LineString(t1)
    line2 = LineString(t2)
    vanishingPoints[key] = line1.intersection(line2).x, line1.intersection(line2).y

print('calculated vanishing points:')
print(vanishingPoints)

In [ ]:
from skimage import transform

#form the projection matrix => scaling factors are missing mult with k_matrix??!!!! ask on wednesday!
vx = (vanishingPoints['x'][0],vanishingPoints['x'][1] ,1)
vy = (vanishingPoints['y'][0],vanishingPoints['y'][1] ,1)
vz = (vanishingPoints['z'][0],vanishingPoints['z'][1] ,1)
w = (0,0,1)

projectionMatrix = np.column_stack([vx,vy,vz,w])

print(projectionMatrix)

In [ ]:
# Extract the camera matrix
camera_matrix = projectionMatrix[:, :3]
# Extract the rotation and translation vectors
rotation_vector, jacobian = cv2.Rodrigues(projectionMatrix[:, :3])
translation_vector = projectionMatrix[:, 3]

# END SECTION MICHI
---